## Importing necessary packages

In [1]:
import time
import random
from typing import List
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

## Loading data into dataframes

The dataset is an excel file with 3 sheets. So each sheet will be loaded as a
dataframe.

In [4]:
# Loading meters table
meter_table = pd.read_excel('gorilla_test_data.xlsx', sheet_name='meter_list')
# displaying all rows
meter_table

,meter_id,aq_kwh,exit_zone
0,14676236,28978,EA1
1,34509937,78324,SO1
2,50264822,265667,NT1
3,88357331,484399,SE2


NOTICE that every **meter ID** is linked to an **exit zone** and also to an 
**annual quantity**.

In [10]:
# meter_table infos
meter_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   meter_id   4 non-null      int64 
 1   aq_kwh     4 non-null      int64 
 2   exit_zone  4 non-null      object
dtypes: int64(2), object(1)
memory usage: 224.0+ bytes


**meter_id** and **aq_kwh** colmns are integers. **exit_zone** is a string.

In [5]:
# Mapping meter ID to exit zone
meter_zone_dict = {row.meter_id: row.exit_zone for row in meter_table.itertuples()}
print(meter_zone_dict)

{14676236: 'EA1', 34509937: 'SO1', 50264822: 'NT1', 88357331: 'SE2'}


In [6]:
# Mapping meter ID to AQ
meter_aq_dict = {row.meter_id: row.aq_kwh for row in meter_table.itertuples()}
print(meter_aq_dict)

{14676236: 28978, 34509937: 78324, 50264822: 265667, 88357331: 484399}


In [3]:
# Loading rate table
rate_table = pd.read_excel('gorilla_test_data.xlsx', sheet_name='rate_table')
# displaying first 5 rows
rate_table.head()

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,EA1,0,73200.0,0.2652
1,2020-04-01,EA1,73200,732000.0,0.1980
2,2020-04-01,EA1,732000,NaN,0.2875
3,2020-04-01,EA2,0,73200.0,0.2970
4,2020-04-01,EA2,73200,732000.0,0.1524


In [8]:
# rate_table infos
rate_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140 entries, 0 to 1139
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            1140 non-null   datetime64[ns]
 1   exit_zone       1140 non-null   object        
 2   aq_min_kwh      1140 non-null   int64         
 3   aq_max_kwh      760 non-null    float64       
 4   rate_p_per_kwh  1140 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 44.7+ KB


In [19]:
# Checking for duplicated rows
rate_table.duplicated().any()

False

In [7]:
# Loading forecast table
forecast_table = pd.read_excel('gorilla_test_data.xlsx', sheet_name='forecast_table')
# displaying first 5 rows
forecast_table.head()

,meter_id,date,kwh
0,14676236,2020-06-01,22.070768
1,14676236,2020-06-02,19.170720
2,14676236,2020-06-03,23.555111
3,14676236,2020-06-04,18.220712
4,14676236,2020-06-05,14.196134


In [16]:
# forecast_table infos
forecast_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3412 entries, 0 to 3411
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   meter_id  3412 non-null   int64         
 1   date      3412 non-null   datetime64[ns]
 2   kwh       3412 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 80.1 KB


Notice that **date** column is a datetime but not a string.

In [18]:
# Check duplicated rows
forecast_table.duplicated().any()

False

## Finding the correct subset of rates corresponding to each meter

For each meter (**meter_id**) we have the correct exit zone and annual quantity.  
In the rate table, the AQ band has been determined with the assumption that : 
**aq_mim_kwh <= AQ < aq_max_kwh**

In [20]:
def get_rates_per_meter(
    meter_id: int,
    meter_zone_dict=meter_zone_dict,
    meter_aq_dict=meter_aq_dict,
    rate_table=rate_table
) -> pd.DataFrame:
    """
    This function finds the correct subset of rates for each meter.
    """
    cond = ((rate_table.exit_zone == meter_zone_dict[meter_id]) & 
            (rate_table.aq_min_kwh <= meter_aq_dict[meter_id]) & 
            (rate_table.aq_max_kwh > meter_aq_dict[meter_id]))
    rate_per_meter_table = rate_table[cond]

    return rate_per_meter_table

In [22]:
rate_per_meter_table = get_rates_per_meter(meter_id=14676236)
rate_per_meter_table

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,EA1,0,73200.0,0.2652
114,2020-10-01,EA1,0,73200.0,0.2970
228,2021-04-01,EA1,0,73200.0,0.3327
342,2021-10-01,EA1,0,73200.0,0.3726
456,2022-04-01,EA1,0,73200.0,0.4173
570,2022-10-01,EA1,0,73200.0,0.4674
684,2023-04-01,EA1,0,73200.0,0.5235
798,2023-10-01,EA1,0,73200.0,0.5863
912,2024-04-01,EA1,0,73200.0,0.6566
1026,2024-10-01,EA1,0,73200.0,0.7354


The rates that determine gas transportation charges generally change twice a 
year in April and October:  
1. **2020-04-01 - 2020-09-30** ----------> 0.2652
2. **2020-10-01 - 2021-03-31** ----------> 0.2970
3. **2021-04-01 - 2021-09-30** ----------> 0.3327
4. **2021-10-01 - 2022-03-31** ----------> 0.3726
5. **2022-04-01 - 2022-09-30** ----------> 0.4173
6. **2022-10-01 - 2023-03-31** ----------> 0.4674
7. **2023-04-01 - 2023-09-31** ----------> 0.5235
8. **2023-10-01 - 2024-03-31** ----------> 0.5863
9. **2024-04-01 - 2024-09-30** ----------> 0.6566
10. **2024-10-01 - 2025-03-31** ----------> 0.7354

## Finding the rate for each meter and day

In [27]:
def get_rate(
    meter_id: int,
    date: datetime
) -> float:
    """
    - This function finds the rate for each meter and day.
    """
    
    rate_per_meter_table = get_rates_per_meter(meter_id)

    if datetime(2020, 4, 1) <= date < datetime(2020, 9, 30):
        return rate_per_meter_table.iloc[0, 4]
    if datetime(2020, 10, 1) <= date < datetime(2021, 3, 31):
        return rate_per_meter_table.iloc[1, 4]
    if datetime(2021, 4, 1) <= date < datetime(2021, 9, 30):
        return rate_per_meter_table.iloc[2, 4]
    if datetime(2021, 10, 1) <= date < datetime(2022, 3, 31):
        return rate_per_meter_table.iloc[3, 4]
    if datetime(2022, 4, 1) <= date < datetime(2022, 9, 30):
        return rate_per_meter_table.iloc[4, 4]
    if datetime(2022, 10, 1) <= date < datetime(2023, 3, 31):
        return rate_per_meter_table.iloc[5, 4]
    if datetime(2023, 4, 1) <= date < datetime(2023, 9, 30):
        return rate_per_meter_table.iloc[6, 4]
    if datetime(2023, 10, 1) <= date < datetime(2024, 3, 31):
        return rate_per_meter_table.iloc[7, 4]
    if datetime(2024, 4, 1) <= date < datetime(2024, 9, 30):
        return rate_per_meter_table.iloc[8, 4]
    if datetime(2024, 10, 1) <= date < datetime(2025, 3, 31):
        return rate_per_meter_table.iloc[9, 4]

In [28]:
get_rate(14676236, datetime(2022, 2, 27))

0.3726

## Adding rate column into forecast table

In [29]:
forecast_table['rate'] = np.vectorize(get_rate, otypes=[float])(forecast_table['meter_id'], forecast_table['date'])

In [30]:
forecast_table.head()

,meter_id,date,kwh,rate
0,14676236,2020-06-01,22.070768,0.2652
1,14676236,2020-06-02,19.170720,0.2652
2,14676236,2020-06-03,23.555111,0.2652
3,14676236,2020-06-04,18.220712,0.2652
4,14676236,2020-06-05,14.196134,0.2652


## Calculating the daily cost

The daily charge is calculated by finding the correct rate for each meter and day 
in the forecast and multiplying this rate (in p/kWh) with the day's forecast (in kWh).

In [31]:
# Calculate the cost per day for each meter
forecast_table['cost_per_day'] = np.vectorize(lambda x,y: x*y, otypes=[float])(forecast_table['kwh'], forecast_table['rate'])

In [32]:
forecast_table.head()

,meter_id,date,kwh,rate,cost_per_day
0,14676236,2020-06-01,22.070768,0.2652,5.853168
1,14676236,2020-06-02,19.170720,0.2652,5.084075
2,14676236,2020-06-03,23.555111,0.2652,6.246815
3,14676236,2020-06-04,18.220712,0.2652,4.832133
4,14676236,2020-06-05,14.196134,0.2652,3.764815


## Calculating the total cost and total consumption

In [33]:
# Calculate the total cost per meter and the total consumption per meter
group_by_meterID = forecast_table.groupby('meter_id')
total_cost_consumption_per_meter = group_by_meterID.agg({'kwh': 'sum', 'cost_per_day': 'sum'}).reset_index()
total_cost_consumption_per_meter.columns = [
    'meter_id', 
    'Total Estimated Consumption [kWh]', 
    'Total cost [p]'
]

In [34]:
# Converting cost in pound
total_cost_consumption_per_meter['Total_cost [£]'] = total_cost_consumption_per_meter['Total cost [p]']*0.01
total_cost_consumption_per_meter['Total_cost [£]'] = np.vectorize(lambda x: round(x, 2), otypes=[float])(total_cost_consumption_per_meter['Total_cost [£]'])
transportation_cost_table = total_cost_consumption_per_meter.drop('Total cost [p]', axis=1)

In [35]:
transportation_cost_table

,meter_id,Total Estimated Consumption [kWh],Total_cost [£]
0,14676236,28978.0,99.45
1,34509937,78324.0,273.83
2,50264822,265667.0,726.47
3,88357331,484399.0,1424.38


# 2. Write a function that generates a list of random meters of any size

In [36]:
def generate_meters(valide_exit_zone: List[str]) -> pd.DataFrame:
    """
    - This function generates a list of random meter IDs and random AQ for every
      exit zone.
    - params:
        - valide_exit_zone: list of valide exit zone.
    """
    
    n = len(valide_exit_zone)
    data = dict()
    
    # generate random meter IDs
    data['meter_id'] = random.sample(range(10000000, 9999999999), n)
    
    # generate random annual quantity
    data['aq_kwh'] = random.sample(range(10000, 999999), n)
    
    # find unique valid exit zone from rate table
    data['exit_zone'] = valide_exit_zone
    
    meter_table = pd.DataFrame(data)
    
    return meter_table

In [38]:
valide_exit_zone = rate_table['exit_zone'].unique().tolist()
generated_meter_table = generate_meters(valide_exit_zone)
generated_meter_table.head(10)

,meter_id,aq_kwh,exit_zone
0,2706922423,217299,EA1
1,7056913858,472857,EA2
2,8433921185,89449,EA3
3,2589774114,404503,EA4
4,8515396460,232587,EM1
5,8147953724,542205,EM2
6,3889611597,337365,EM3
7,4607814367,721555,EM4
8,4632979955,679662,LC
9,9356474222,22597,LO


# 3. Write a function that generates mock consumption data given a list of meters

In [42]:
def mock_consumption_data(
    meter_list: List[int], 
    start_date: datetime, 
    duration: int
) -> pd.DataFrame:
    """
    - This function generates mock consumption data given a list of meters
    - It returns a dataframe
    - params:
        - meter_list: a list of meter IDs
        - start_date: a start date for the forecast period
        - duration: the number of days for the forecast
    """
    # Initializing a list of dataframe
    dataframe_list = []
    
    for meter_id in meter_list:
        data = dict()
        data['meter_id'] = [meter_id]*duration
        
        # Generating every single date corresponding to the forecast duration
        data['date'] = [start_date + timedelta(days=k) for k in range(duration)]
        
        # Generating random consumption for the duration of the forecast period
        data['kwh'] = [round(float_num,3) for float_num in np.random.uniform(10.01, 599.99, duration).tolist()]
        
        dataframe_list.append(pd.DataFrame(data))
        
    # We have a list of dataframe for every meter. We now need to stack 
    # vertically all dataframes into one dataframe
    stacked_dataframe = pd.concat(dataframe_list, axis=0).reset_index()
    stacked_dataframe.drop('index', axis=1, inplace=True)
        
    return stacked_dataframe

In [43]:
meter_list = generated_meter_table.meter_id.to_list()
start_date = datetime(2021, 4, 1)
duration = 386
print(meter_list)

[2706922423, 7056913858, 8433921185, 2589774114, 8515396460, 8147953724, 3889611597, 4607814367, 4632979955, 9356474222, 9863596273, 6221434629, 4321066516, 9930396557, 7345615924, 939161031, 3676155799, 9015788705, 4866724325, 5165204784, 1345670261, 6665326306, 2787665953, 963569546, 2145548558, 91608379, 7277078124, 5173945400, 4666217995, 3311505052, 2737008008, 2785424315, 6370483463, 1423370654, 1699809248, 8615765341, 8313781685, 1177996773]


In [44]:
forecast_data = mock_consumption_data(meter_list, start_date, duration)
forecast_data.head(10)

,meter_id,date,kwh
0,2706922423,2021-04-01,83.136
1,2706922423,2021-04-02,42.310
2,2706922423,2021-04-03,501.058
3,2706922423,2021-04-04,523.324
4,2706922423,2021-04-05,496.964
5,2706922423,2021-04-06,297.639
6,2706922423,2021-04-07,291.346
7,2706922423,2021-04-08,554.638
8,2706922423,2021-04-09,471.351
9,2706922423,2021-04-10,332.462


## 4. Write a function that takes as an input a meter list and a consumption forecast table and that calculates the transportation cost table

In [45]:
def find_rate(
    meter_id: int,
    date: datetime,
    meter_zone_dict: dict,
    meter_aq_dict: dict,
    rate_table: pd.DataFrame
) -> float:
    
    cond = ((rate_table.exit_zone == meter_zone_dict[meter_id]) & 
            (rate_table.aq_min_kwh <= meter_aq_dict[meter_id]) & 
            (rate_table.aq_max_kwh > meter_aq_dict[meter_id]))
    rate_per_meter_table = rate_table[cond]

    if datetime(2020, 4, 1) <= date < datetime(2020, 9, 30):
        return rate_per_meter_table.iloc[0, 4]
    if datetime(2020, 10, 1) <= date < datetime(2021, 3, 31):
        return rate_per_meter_table.iloc[1, 4]
    if datetime(2021, 4, 1) <= date < datetime(2021, 9, 30):
        return rate_per_meter_table.iloc[2, 4]
    if datetime(2021, 10, 1) <= date < datetime(2022, 3, 31):
        return rate_per_meter_table.iloc[3, 4]
    if datetime(2022, 4, 1) <= date < datetime(2022, 9, 30):
        return rate_per_meter_table.iloc[4, 4]
    if datetime(2022, 10, 1) <= date < datetime(2023, 3, 31):
        return rate_per_meter_table.iloc[5, 4]
    if datetime(2023, 4, 1) <= date < datetime(2023, 9, 30):
        return rate_per_meter_table.iloc[6, 4]
    if datetime(2023, 10, 1) <= date < datetime(2024, 3, 31):
        return rate_per_meter_table.iloc[7, 4]
    if datetime(2024, 4, 1) <= date < datetime(2024, 9, 30):
        return rate_per_meter_table.iloc[8, 4]
    if datetime(2024, 10, 1) <= date < datetime(2025, 3, 31):
        return rate_per_meter_table.iloc[9, 4]

In [46]:
def calculate_transportation_cost(
    rate_table: pd.DataFrame,
    start_date: datetime,
    duration: int
) -> pd.DataFrame:
    """
    - This function takes as input: meter_list and forecast_table.
    - returns the transportation cost table: task1's final result.
    """
    start = time.perf_counter()

    # generate meters
    valide_exit_zone = rate_table['exit_zone'].unique().tolist()
    generated_meter_table = generate_meters(valide_exit_zone)
    meter_list = generated_meter_table.meter_id.to_list()

    # Mapping meter ID to exit zone
    meter_zone_dict = {row.meter_id: row.exit_zone for row in generated_meter_table.itertuples()}
    
    # Mapping meter ID to AQ
    meter_aq_dict = {row.meter_id: row.aq_kwh for row in generated_meter_table.itertuples()}

    # consumption data
    forecast_data = mock_consumption_data(meter_list, start_date, duration)
    
    # Adding rate to forecast table
    forecast_data['rate'] = np.vectorize(find_rate, otypes=[float])(forecast_data['meter_id'], forecast_data['date'])
    
    # Calculate the cost per day for each meter
    forecast_data['cost_per_day'] = np.vectorize(lambda x,y: x*y, otypes=[float])(forecast_data['kwh'], forecast_data['rate_per_day'])
    
    # Calculate the total cost per meter and the total consumption per meter
    group_by_meterID = forecast_data.groupby('meter_id')
    total_cost_consumption_per_meter = group_by_meterID.agg({'kwh': 'sum', 'cost_per_day': 'sum'}).reset_index()
    total_cost_consumption_per_meter.columns = [
        'meter_id', 
        'Total Estimated Consumption [kWh]', 
        'Total cost [p]'
    ]
    
    # Converting cost in pound
    total_cost_consumption_per_meter['Total_cost [£]'] = total_cost_consumption_per_meter['Total cost [p]']*0.01
    total_cost_consumption_per_meter['Total_cost [£]'] = np.vectorize(lambda x: round(x, 2), otypes=[float])(total_cost_consumption_per_meter['Total_cost [£]'])
    transportation_cost_table = total_cost_consumption_per_meter.drop('Total cost [p]', axis=1)
                                                                                                                       
    stop = time.perf_counter()
    print(f"Completed Execution in {stop - start} seconds\n")
    
    return transportation_cost_table

In [ ]:
transportation_cost = calculate_transportation_cost(rate_table, datetime(2021, 4, 1), 386)